# A Model of the Reduction of HIV Concentration During Acute Infection

Kristtiya Vachiraseneekul Guerra

In [20]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [21]:
def make_system():
    """Make a system object for the HIV model.

    gamma: rate of new uninfected CD4
    mu: HIV-independent Death Rate
    tau: Proportion activated
    delta: rate of removal of infected cells
    sigma: rate of removal of free virions
    beta: Infection Rate
    rho: proportion of cells becoming latently infected
    alpha = activation rate
    
    returns: System object
    """
    init = State(R=99, L=0, V=1, E=0)
    init /= np.sum(init)

    t0 = 0
    t_end = 2000
    gamma = 1.36
    tau = 0.2
    mu = 0.00136
    beta = 0.00027
    rho = 0.1
    alpha = 0.036
    delta = 0.33
    pi = 100
    sigma = 2
    dt = 0.1
    time_steps = 200

    return System(init=init, t0=t0, t_end=t_end,
                  beta=beta, gamma=gamma, mu=mu, tau=tau,
                delta= delta, sigma=sigma, rho=rho, alpha=alpha, pi=pi, dt=dt, time_steps=time_steps)


In [22]:
system = make_system()

,values
init,R 0.99 L 0.00 V 0.01 E 0.00 dtype:...
t0,0
t_end,2000
beta,0.00027
gamma,1.36
mu,0.00136
tau,0.2
delta,0.33
sigma,2
rho,0.1


In [23]:
def update_func(state, t, system):
    """Update the HIV model.
    
    state: State(r, l, v, e)
    t: time
    system: System object
    
    returns: State (rlve)
    """
    
    unpack(system)
    r, l, v, e = state
  

    infected = ((system.gamma * system.rho * r * v) - (system.mu * l) - (system.alpha * l))
    activated = ((system.gamma * system.tau) - (system.mu * r) - (system.beta * r * v))
    zer = (((1 - system.rho) * system.beta * r * v) + (system.alpha * l) - (system.delta * e)) 
    wow = (system.pi * e - system.rho * v)
    
    
    r += activated * dt
    l += infected * dt
    v += wow * dt
    e += zer * dt
    
    return State(R=r, L=l, V=v, E=e)

In [24]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    
    
    frame = TimeFrame(columns=init.index)
    frame.row[t0] = init
    
    for t in linrange(t0, t_end):
        frame.row[t+1] = update_func(frame.row[t], t, system)
    
    return frame


In [25]:
results = run_simulation(system, update_func)
plot_results(results.V, 'hippo')

TypeError: plot_results() takes 0 positional arguments but 2 were given